# LangChain と Gemini API で PDF ドキュメントの要約を行う例

LangChain のパッケージ、および、PDF の扱いに必要なパッケージをインストールします。

In [ ]:
!pip install --user \
  langchain==0.3.27 transformers==4.36.0 \
  pypdf==6.0.0 cryptography==42.0.4 \
  langchain-google-vertexai==2.0.28 \
  google-cloud-aiplatform==1.111.0 \
  langchain-community==0.3.29

**注意：次のセルを実行する前にカーネルをリスタートしてください。**

NISC が一般公開している「[インターネットの安全・安心ハンドブック](https://security-portal.nisc.go.jp/guidance/handbook.html)」のプロローグ部分をダウンロードします。





In [1]:
base_url = 'https://raw.githubusercontent.com/google-cloud-japan/sa-ml-workshop/main'
!wget -q $base_url/genAI_book/PDF/handbook-prologue.pdf

ダウンロードした PDF を読み込んで、全体を1つのテキストにまとめます。

In [2]:
from langchain_community.document_loaders import PyPDFLoader

pages = PyPDFLoader('handbook-prologue.pdf').load()
document = ''
for page in pages:
    document += page.page_content

Gemini API を利用するクライアントオブジェクトを取得します。

In [3]:
from langchain_google_vertexai import VertexAI
llm = VertexAI(model_name='gemini-2.5-flash-lite', location='us-central1',
               temperature=0.1, max_output_tokens=1024)

RecursiveCharacterTextSplitter, load_qa_chain, AnalyzeDocumentChain モジュールの使用例です。

`input_document` に与えたテキストに基づいて、`question` の質問に回答します。

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import AnalyzeDocumentChain

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=6000, chunk_overlap=200)
qa_chain = load_qa_chain(llm, chain_type='map_reduce')
qa_document_chain = AnalyzeDocumentChain(
    combine_docs_chain=qa_chain, text_splitter=text_splitter)

output = qa_document_chain.invoke(
    {'input_document':'今は６月で雨が多い時期です。', 'question':'最近の天候は？'})
print(output)

/var/tmp/ipykernel_38209/965804757.py:7: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  qa_chain = load_qa_chain(llm, chain_type='map_reduce')
/var/tmp/ipykernel_38209/965804757.py:8: LangChainDeprecationWarning: The class `AnalyzeDocumentChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~example in API reference with more detail: https://api.python.langchain.com/en/latest/chains/langchain.chains.combine_documents.base.A

{'input_document': '今は６月で雨が多い時期です。', 'question': '最近の天候は？', 'output_text': '最近は雨が多いです。'}


`input_document` に与えたテキストの内容をまとめるように指示して結果を得る関数を用意します。

In [5]:
def get_description(document):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=6000, chunk_overlap=200)
    qa_chain = load_qa_chain(llm, chain_type='map_reduce')
    qa_document_chain = AnalyzeDocumentChain(
        combine_docs_chain=qa_chain, text_splitter=text_splitter)

    prompt = '何についての文書ですか？日本語で200字程度にまとめて教えてください。'
    description = qa_document_chain.invoke(
        {'input_document': document, 'question': prompt})
    return description['output_text'].replace('FINAL ANSWER: ', '')

先に用意した PDF ファイルを適用します。

In [6]:
print(get_description(document))

この文書は、インターネットの安全と安心に関するハンドブックの一部であり、サイバー攻撃の基本的なリスク、トラブル、およびそれらへの対策について解説しています。サイバー攻撃の定義、攻撃者の種類、マルウェア（コンピュータウイルスなど）の種類や機能、フィッシング詐欺、ランサムウェア、DDoS攻撃、BEC（ビジネスメール詐欺）といった具体的な攻撃手法について説明しています。また、人間の心理を突くソーシャルエンジニアリングの危険性や、SNSでの誹謗中傷、なりすまし、ネットいじめなどのトラブルにも触れています。さらに、パスワード管理、OSやソフトウェアのアップデート、多要素認証などの基本的なセキュリティ対策の重要性を強調し、サイバー攻撃から身を守るための9か条を提示しています。全体として、インターネットを安全に利用するための知識と意識向上を目的としています。


任意の質問に答える関数を用意します。

In [7]:
def get_answer(document, question):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=4000, chunk_overlap=200)
    qa_chain = load_qa_chain(llm, chain_type='refine')
    qa_document_chain = AnalyzeDocumentChain(
        combine_docs_chain=qa_chain, text_splitter=text_splitter)

    prompt = '{} 日本語で200字程度にまとめて教えてください。マークダウンを使用せずにプレーンテキストで出力。'.format(question)
    answer = qa_document_chain.invoke(
        {'input_document': document, 'question': prompt})
    return answer['output_text']

先に用意した PDF ファイルに基づいて、質問の回答を取得します。

In [11]:
question = 'サイバーセキュリティ対策のポイント９か条を箇条書きにまとめてください。'
print(get_answer(document, question))

サイバーセキュリティ対策のポイント9か条は以下の通りです。

1.  心当たりのないメールの添付ファイルやリンクに注意する。
2.  OSやソフトウェアは常に最新の状態に保つ。
3.  パスワードは長く複雑にし、使い回さない。
4.  多要素認証を利用してログインを安全にする。
5.  偽メールや偽サイトに騙されないよう用心する。
6.  スマホやPCの画面ロックを利用する。
7.  大切な情報は失う前にバックアップする。
8.  外出先では紛失・盗難・覗き見に注意する。
9.  困ったときは一人で悩まず、相談窓口に相談する。
